In [6]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import urllib
import math
import copy
import pandas as pd
import numpy as np

In [14]:
# Determine the desired season start, end, number of games, and from or to the end date
start_year = "2017"
end_year = "2018"
from_to = "From"
games = 82

if(from_to == "From"):
    from_to_html = 1
else:
    from_to_html =-1

In [15]:
# Create a parser to scrape table rows only (no headers) from Puck On Net

class HTMLTableParser:

    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        return [(table['id'],self.parse_html_table(table))\
                for table in soup.find_all('table')]  

    def parse_html_table(self, table):
        n_columns = 0
        n_rows=0
        column_names = []

        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):

            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)


        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                          index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1

        return df

In [16]:
# ADVANCED 1
# Create an empty dataframe with the column headers for missed shots

advanced_df = pd.DataFrame(columns=['Team', 'Game', 'Points',
                                'Corsi S Adj', 'Corsi SV Adj', 'Corsi EVA Adj', 'Corsi Close', 
                                'Fenwick S Adj', 'Fenwick SV Adj', 'Fenwick EVA Adj', 'Fenwick Close', 
                                'SOG S Adj', 'SOG SV Adj', 'SOG EVA Adj', 'SOG Close'
                                ])

In [17]:
# ADVANCED 2
# Loop through tables for the cumulative totals after each w regular-season games in the season requested

w=0

for w in range(games):

    advanced_url = "http://www.puckon.net/index.php?s="+start_year+"-09-01&e="+end_year+"-06-30&f=0&ld="+str(from_to_html)+"&l="+str(w+1)+"&p=0"
    hp = HTMLTableParser()
    table = hp.parse_url(advanced_url)[0][1] # Grabbing the table from the tuple
    table.columns = advanced_df.columns
    advanced_df = pd.concat([table, advanced_df], axis=0, ignore_index=True)

    w = w+1
    

advanced_stats = advanced_df
advanced_stats.head(32)

,Team,Game,Points,Corsi S Adj,Corsi SV Adj,Corsi EVA Adj,Corsi Close,Fenwick S Adj,Fenwick SV Adj,Fenwick EVA Adj,Fenwick Close,SOG S Adj,SOG SV Adj,SOG EVA Adj,SOG Close
0,ANA,82,101,49.00,48.99,49.77,48.13,48.98,48.96,50.00,48.35,48.46,48.49,50.51,47.81
1,ARI,82,70,47.85,47.81,47.25,47.95,46.55,46.54,45.84,46.63,47.38,47.42,46.37,47.48
2,BOS,82,112,53.76,53.83,54.05,52.29,53.66,53.71,53.99,52.30,53.50,53.57,54.19,52.26
3,BUF,82,62,46.45,46.44,44.57,47.33,46.50,46.47,44.10,47.26,47.63,47.62,43.59,48.28
4,CAR,82,83,54.58,54.53,53.03,55.09,53.38,53.34,51.40,54.23,53.91,53.92,50.31,53.92
5,CBJ,82,97,51.66,51.63,51.95,51.20,52.60,52.59,53.00,52.24,52.19,52.20,52.80,51.76
6,CGY,82,84,53.06,53.01,51.97,52.23,52.85,52.80,51.40,52.13,52.59,52.59,49.74,51.92
7,CHI,82,76,52.34,52.32,51.64,51.24,50.84,50.86,49.90,49.67,51.20,51.25,49.54,49.65
8,COL,82,95,47.87,47.89,48.52,48.28,47.27,47.33,48.22,47.43,47.19,47.26,48.85,47.28
9,DAL,82,92,51.33,51.28,51.63,51.43,51.43,51.42,51.87,51.39,51.32,51.33,52.17,51.22


In [18]:
# Add a column to specify the season's ending year, and a column to specify regular season or playoffs
advanced_stats['Season End'] = end_year
advanced_stats['Season Type'] = 'Regular'
advanced_stats['From / To'] = from_to
advanced_stats.head()

,Team,Game,Points,Corsi S Adj,Corsi SV Adj,Corsi EVA Adj,Corsi Close,Fenwick S Adj,Fenwick SV Adj,Fenwick EVA Adj,Fenwick Close,SOG S Adj,SOG SV Adj,SOG EVA Adj,SOG Close,Season End,Season Type,From / To
0,ANA,82,101,49.00,48.99,49.77,48.13,48.98,48.96,50.00,48.35,48.46,48.49,50.51,47.81,2018,Regular,From
1,ARI,82,70,47.85,47.81,47.25,47.95,46.55,46.54,45.84,46.63,47.38,47.42,46.37,47.48,2018,Regular,From
2,BOS,82,112,53.76,53.83,54.05,52.29,53.66,53.71,53.99,52.30,53.50,53.57,54.19,52.26,2018,Regular,From
3,BUF,82,62,46.45,46.44,44.57,47.33,46.50,46.47,44.10,47.26,47.63,47.62,43.59,48.28,2018,Regular,From
4,CAR,82,83,54.58,54.53,53.03,55.09,53.38,53.34,51.40,54.23,53.91,53.92,50.31,53.92,2018,Regular,From


In [19]:
# Export file as a CSV, without the Pandas index, but with the header
csv_write_path = "Desktop/nhl_predictor/advanced_stats"+end_year+"_"+from_to+"-season_end.csv"
advanced_stats.to_csv(csv_write_path, index=False, header=True)